# Generate CMOD dataset

In [57]:
import os
os.getcwd()

'/home/yumouwei/2025_wm_summer_school/2025-wm-summer-school-data'

In [ ]:
'''# read shotlists
import pandas as pd
# Identified shots that cause critical error 
ERROR_SHOTS = [1120215001, 1120801004, 1120801017, 1120807021, 1120816012, 
               1120831018, 1120914031, 1120807019, 1120918013, 1150528006]

disruption_shotlist = pd.read_csv('./shotlists/disruption_shotlist.txt', header=None)
disruption_shotlist = disruption_shotlist[0].astype(int).to_list()
failed_disruption_shotlist = pd.read_csv('./shotlists/disruption_shotlist_failed.txt', header=None)
failed_disruption_shotlist = failed_disruption_shotlist[0].astype(int).to_list()
non_disruptive_shotlist = pd.read_csv('./shotlists/non_disruptive_shotlist.txt', header=None)
non_disruptive_shotlist = non_disruptive_shotlist[0].astype(int).to_list()

print(f"Number of disruptive shots: {len(disruption_shotlist)}")
print(f"Number of failed disruptive shots: {len(failed_disruption_shotlist)}")
print(f"Number of non disruptive shots: {len(non_disruptive_shotlist)}")

# Combine the disruptive and non-disruptive shotlists, then remove the failed shots
cmod_shotlist = disruption_shotlist + non_disruptive_shotlist
cmod_shotlist.sort()

for failed_shot in failed_disruption_shotlist + ERROR_SHOTS:
    cmod_shotlist.remove(failed_shot)

print(f"Total number of shots to be fetched: {len(cmod_shotlist)}")'''

Number of disruptive shots: 3795
Number of failed disruptive shots: 74
Number of non disruptive shots: 6640
Total number of shots to be fetched: 10351


In [ ]:
import pandas as pd

cmod_shotlist = pd.read_csv('./shotlists/cmod_shotlist.txt', header=None)
cmod_shotlist = cmod_shotlist[0].astype(int).to_list()
print(f"Total number of shots to be fetched: {len(cmod_shotlist)}")

Total number of shots to be fetched: 10351


## Generate datasets

In [ ]:
from disruption_py.settings import LogSettings, RetrievalSettings
from disruption_py.workflow import get_shots_data

csv_filename = "./data/cmod_dataset.csv"

retrieval_settings = RetrievalSettings(
    time_setting = "disruption_warning",
)

out = get_shots_data(
    shotlist_setting = cmod_shotlist,
    tokamak = "cmod",
    retrieval_settings = retrieval_settings,
    output_setting = csv_filename,
    num_processes = 20,    # Use 20 processors
    log_settings=LogSettings(  # logging
        log_file_path=None,  # Defaults to "output.log" in the tmp folder for the session
        file_log_level="SUCCESS",
        log_file_write_mode="w",
        log_to_console=True,
        use_custom_logging=False,
    )
)

21:06:24.629 [ INFO  ] Starting: disruption_py ~ v0.10.0 # 89c7dc5 / yumouwei@mferws04


  0%|          | 0/10351 [00:00<?, ?it/s]

21:06:26.111 [WARNING] #1050204013 | get_ts_parameters: Failed! %TREE-E-NODATA, No data available. Tree: electrons, Node: .yag_new.results.profiles:te_rz
21:06:26.444 [WARNING] #1050208001 | get_n_equal_1_amplitude: Failed! %TREE-E-NODATA, No data available. Tree: magnetics, Node: \mag_bp_coils.signals.BP13_GH
21:06:26.453 [WARNING] #1050207015 | get_n_equal_1_amplitude: Failed! %TREE-E-NODATA, No data available. Tree: magnetics, Node: \mag_bp_coils.signals.BP13_GH
21:06:26.484 [WARNING] #1050208017 | get_n_equal_1_amplitude: Failed! %TREE-E-NODATA, No data available. Tree: magnetics, Node: \mag_bp_coils.signals.BP13_GH
21:06:26.503 [WARNING] #1050208019 | get_n_equal_1_amplitude: Failed! %TREE-E-NODATA, No data available. Tree: magnetics, Node: \mag_bp_coils.signals.BP13_GH
21:06:26.532 [WARNING] #1050208015 | get_n_equal_1_amplitude: Failed! %TREE-E-NODATA, No data available. Tree: magnetics, Node: \mag_bp_coils.signals.BP13_GH
21:06:26.583 [WARNING] #1050207007 | get_n_equal_1_ampli

In [ ]:
# check dumped csv matches out
from pandas.testing import assert_frame_equal

saved = pd.read_csv(csv_filename)
try:
    assert_frame_equal(out, saved[out.columns])
    print('Success!')
except AssertionError:
    print('ERROR: Dumped csv file does not match the fetched dataset!')